In this notebook, we train a CNN model to classify images from the CIFAR-10 database. 

The images are colored images and fall into 10 categories.

In [1]:
## Importing libraries
import torch
import numpy as np

# Libraries for data loading
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler



In [2]:
#Test for CUDA
## Cuda uses GPU and can speed up the training process. Using CUDA can increase the speed in which the model
## trains
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("CUDA is available")
else:
    print("CUDA is not available")

CUDA is not available


#### Loadind the Data
CIFAR is part of pytorch library so we can download the data and split it into train and test dataset. We will create 
a dataloader for each of these datasets. 

In [5]:
#Number of subprocess to use
num_workers = 0

#Number samples per batch
batch_size = 20

#percentage training set
valid_size = 0.2

# Transform is defined in PyTorch to transform the given data another format.
# Some examples are transforming from numpy array to tensor etc.
# Transform compose accepts a list in which we can pass multiple transformations. 
# Refer to this youtube tutorial for more info - https://www.youtube.com/watch?v=X_QOZEko5uE 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [6]:
# Getting training and test Dataset
train_data = datasets.CIFAR10('data', train=True, download = True, transform = transform)
test_data = datasets.CIFAR10('data', train = True, download = True, transform = transform)

# Getting indices and performing training split and validation split
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

Files already downloaded and verified
Files already downloaded and verified
